### Cab-Driver Agent

In [1]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

# for plotting graphs
import matplotlib.pyplot as plt

# Import the environment
from Env import CabDriver
import os
import collections


f:\installedsoftware\anaconda\envs\rasa\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Defining Time Matrix

In [2]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")

#### Tracking the state-action pairs for checking convergence


In [3]:
Q_dict = collections.defaultdict(dict)

States_track = collections.defaultdict(dict)

rewards_tracked = {}
for i in range(0,5):
    for j in range(0,24):
        for k in range(0,7):
            rewards_tracked[(i,j,k)]=[]


print(len(Q_dict))
print(len(rewards_tracked))
print(len(States_track))


0
840
0


In [4]:
def save_tracking_states():
    for state in States_track.keys():
        for action in States_track[state].keys():
            if state in Q_dict and action in Q_dict[state]:
                States_track[state][action].append(Q_dict[state][action]) 

In [5]:
total_actions=[(0,0)]
for i in range(0,5):
    for j in range(0,5):
        if i!=j:
            total_actions.append((i,j))

total_state = [(i,j,k) for i in range(0,5) for j in range(0,24) for k in range(0,7)]

for state in total_state:
    Q_dict[state] = {}
    for action in total_actions:
        Q_dict[state][action] = 0.0
        

In [6]:
def initialise_tracking_states():
    for state in total_state:
        for action in total_actions:
            States_track[state][action] = [] 

In [7]:
def find_action_indx(val):
    for i,item in enumerate(total_actions):
        if val == item:
            return i
    return 0

def convert_into_str(arr):
    text=""
    for val in arr:
        text=text+"-"+str(int(val))
    return text



In [8]:
Q_dict[(0,0,0)][0,1]

0.0

In [9]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state-action and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [10]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor = 0.75
        self.learning_rate =  0.01    
        self.epsilon_max = 0.99
        self.epsilon_decay = -0.00005
        self.epsilon_min = 0.01
        self.batch_size = 32
        self.epsilon = 1
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()


    # approximate Q function using Neural Network
    def build_model(self):
        input_shape=self.state_size
        model = Sequential()
        # Write your code here: Add layers to your neural nets
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))

        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))
        
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model



    def get_action(self, state, action):
        # Write your code here:
        # get action from model using epsilon-greedy policy
        # Decay in ε after we generate each sample from the environment
        if np.random.rand() <= self.epsilon:
            return action[random.randrange(self.action_size)]
        else:
            state = state.reshape(1, self.state_size)
            q_value = self.model.predict(state)
            return total_actions[np.argmax(q_value[0])]

        
    

    def append_sample(self, state, action, reward, next_state):
        # Write your code here:
        # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state))
    

    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self,terminal_state):
        if len(self.memory) > self.batch_size:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            update_output = np.zeros((self.batch_size, self.state_size))
            update_input = np.zeros((self.batch_size, self.state_size))
            actions, rewards = [], []
            
            for i in range(self.batch_size):
                state, action, reward, next_state = mini_batch[i]
                state_encod = env.state_encod_arch2(state,action)
                
                update_input[i] = state_encod
                actions.append(action)
                rewards.append(reward)
                update_output[i] = env.state_encod_arch2(next_state,(0,0))
            
            target = self.model.predict(update_input)

            target_qval = self.model.predict(update_output)
            for i in range(self.batch_size):
                if terminal_state:
                    target[i][find_action_indx(actions[i])] = rewards[i]
                else: # non-terminal state
                    target[i][find_action_indx(actions[i])] = rewards[i] + self.discount_factor * np.max(target_qval[i])

            # model fit
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)
                
        # 4. Fit your model and track the loss values



    def save(self, name):
        self.model.save_weights(name)

In [12]:
Episodes = 20000

### DQN block

In [ ]:
env = CabDriver()
state_size = 5+7+24+5+5 # equal to 4 in case of cartpole 
action_size = len(env.action_space)            # equal to 2 in case of cartpole
agent= DQNAgent(state_size,action_size)
rewards_per_episode, episodes = [], []

LR = agent.learning_rate
GAMMA = agent.discount_factor
threshold = 1000
policy_threshold = 1000
if not os.path.exists("saved_model_weights"):
    os.mkdir("saved_model_weights")
    
for episode in range(10000,Episodes):
    
    score = 0
    time_stamp=0
    action_space,state_space,state = env.reset()
    # Write code here
    # Call the environment
    # Call all the initialised variables of the environment
#     agent.epsilon = - 1/ (1 + np.exp((-episode+7500000)/17000000)) + 1
    agent.epsilon = (1 - 0.00001) * np.exp(agent.epsilon_decay * episode)
    
    initial_state = env.state_init
    #Call the DQN agent
    terminal_state=False
    while not terminal_state:
#         agent.epsilon = agent.epsilon_min + (agent.epsilon_max - agent.epsilon_min) * np.exp(-agent.epsilon_decay*episode)
        if time_stamp > 30*24:
            terminal_state=True;
        z = np.random.random()
        time_stamp+=1
        action = env.requests(state)[1]
        agent.action_size=len(action)
        take_action=agent.get_action(env.state_encod_arch2(state,(0,0)),action)
        next_state = env.next_state_func(state,take_action,Time_matrix)
        reward= env.reward_func(state,take_action,Time_matrix)
        agent.append_sample(state, take_action, reward, next_state)
        max_next = max(Q_dict[next_state],key=Q_dict[next_state].get)
        Q_dict[state][take_action] += LR * ((reward + (GAMMA*(Q_dict[next_state][max_next]))) - Q_dict[state][take_action] ) 
            

            
        score += reward
        state = next_state

        agent.train_model(terminal_state)
        
    
        
    rewards_per_episode.append(score)
    episodes.append(episode)
    

        
    print("episode {0}, reward {1}, memory_length {2}, epsilon {3}".format(episode,
                                                                         score,
                                                                         len(agent.memory),
                                                                         agent.epsilon))
    if episode % 10 == 0:
        # store q-values of some prespecified state-action pairs
        # q_dict = agent.store_q_values()

        # save model weights
        agent.save(name="model_weights.h5")

    if initial_state in rewards_tracked:     #storing rewards
        rewards_tracked[initial_state].append(score)
        #save_obj(rewards_tracked,'Rewards')

    if ((episode+1) % threshold) == 0:   #every 2000th episode
        save_obj(rewards_tracked,'Rewards')   
    
    #TRACKING Q-VALUES
    if (episode == threshold+10000):      #at the 1999th episode
        print("initializing initialise_tracking_states.........................")
        initialise_tracking_states()
      
    if ((episode+1) % threshold) == 0:   #every 2000th episode
        save_tracking_states()
        save_obj(States_track,'States_tracked')   
    
    #SAVING POLICY
    if ((episode+1)% policy_threshold ) == 0:  #every 30000th episodes, the Q-dict will be saved
        save_obj(Q_dict,'Policy')    
        
        
save_obj(rewards_tracked,'Rewards')   
save_obj(States_track,'States_tracked')   
save_obj(Q_dict,'Policy')      
print(episode)
    
    

episode 10000, reward 689.0, memory_length 722, epsilon 0.6065245944060363
episode 10001, reward 1990.0, memory_length 1444, epsilon 0.6064942689344591
episode 10002, reward 1873.0, memory_length 2000, epsilon 0.6064639449791176
episode 10003, reward 2287.0, memory_length 2000, epsilon 0.6064336225399359
episode 10004, reward 1193.0, memory_length 2000, epsilon 0.6064033016168383
episode 10005, reward 1072.0, memory_length 2000, epsilon 0.606372982209749
episode 10006, reward 1556.0, memory_length 2000, epsilon 0.6063426643185921
episode 10007, reward 1673.0, memory_length 2000, epsilon 0.6063123479432918
episode 10008, reward 571.0, memory_length 2000, epsilon 0.6062820330837725
episode 10009, reward 660.0, memory_length 2000, epsilon 0.6062517197399583
episode 10010, reward 881.0, memory_length 2000, epsilon 0.6062214079117733
episode 10011, reward 1215.0, memory_length 2000, epsilon 0.6061910975991418
episode 10012, reward 1614.0, memory_length 2000, epsilon 0.6061607888019881
episo

episode 10107, reward 2840.0, memory_length 2000, epsilon 0.6032883524921909
episode 10108, reward 2118.0, memory_length 2000, epsilon 0.6032581888286643
episode 10109, reward 2188.0, memory_length 2000, epsilon 0.603228026673283
episode 10110, reward 1627.0, memory_length 2000, epsilon 0.6031978660259718
episode 10111, reward 1524.0, memory_length 2000, epsilon 0.6031677068866553


In [ ]:
env.state_space

### Tracking Convergence

In [ ]:
with open('Policy.pkl', 'rb') as handle:
    Q_dict = pickle.load(handle)
    
with open('Rewards.pkl', 'rb') as handle:
    rewards_tracked = pickle.load(handle)    

with open('States_tracked.pkl', 'rb') as handle:
    States_track = pickle.load(handle)    
    
print(len(Q_dict))
print(len(rewards_tracked))
print(len(States_track))
States_track

In [15]:
States_track

defaultdict(dict, {})

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))

In [ ]:
plt.plot(time, epsilon)
plt.show()

In [ ]:
env = CabDriver()
state_size = env.state_space.shape[0]

In [ ]:
state_size

In [ ]:
np.argmax([10,14,15,45])